# 计算地址在代币上的收益

```sql
WITH rlzd AS (
    SELECT
        trader_address,
        SUM(eth_price) AS pnl_eth
    FROM (
             SELECT
                 origin_address AS trader_address,
                 SUM(-quantity_in / 1e18) AS eth_price
             FROM ethereum.dex_swaps ds
                      LEFT JOIN ethereum.contract_labels cl ON cl.contract_address = origin_address
             WHERE from_token_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
               AND to_token_address = '{{token_address}}'
               AND cl.labels ISNULL
             GROUP BY 1
             UNION ALL
             SELECT
                 origin_address AS trader_address,
                 SUM(quantity_out / 1e18) AS eth_price
             FROM ethereum.dex_swaps ds
                      LEFT JOIN ethereum.contract_labels cl ON cl.contract_address = origin_address
             WHERE from_token_address = '{{token_address}}'
               AND to_token_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
               AND cl.labels ISNULL
             GROUP BY 1
         ) s
    GROUP BY 1
), unrlzd AS (
    SELECT
        owner_address AS trader_address,
        (balance / 10 ^ decimals) * tok_price_usd / eth_price_usd AS unrealized_eth_price
    FROM ethereum.token_owners own
             NATURAL JOIN (SELECT close_price AS tok_price_usd FROM ethereum.token_prices_ohlc_1d ohlc
                           WHERE token_address = '{{token_address}}'
                           ORDER BY timestamp DESC LIMIT 1) tok_price
             NATURAL JOIN (SELECT close_price AS eth_price_usd FROM ethereum.token_prices_ohlc_1d ohlc
                           WHERE token_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
                           ORDER BY timestamp DESC LIMIT 1) eth_price
             NATURAL JOIN (SELECT decimals FROM ethereum.tokens t WHERE contract_address = '{{token_address}}') tok_decimals
    LEFT JOIN ethereum.contract_labels cl ON cl.contract_address = owner_address
    WHERE own.contract_address = '{{token_address}}'
        AND cl.labels ISNULL
)
SELECT
     rlzd.*,
     unrlzd.unrealized_eth_price AS unrealized_pnl_eth
FROM rlzd
LEFT JOIN unrlzd USING(trader_address)
ORDER BY pnl_eth DESC;
```